# Advent of Code 2021 Solutions

In [1]:
import pandas as pd
import numpy as np
from aoc import get_input_data

---
## Day 1

### part 1

In [2]:
data = get_input_data(1, year=2021)
data = list(map(int, data.split('\n')))

prev = np.inf
count = 0

for x in data:
    if x > prev:
        count += 1
    prev = x
count

1121

### part 2

In [3]:
prev = np.inf
count = 0
stride = 3

for i in range(len(data) - stride + 1):
    depths = data[i:i + stride]
    total = sum(depths)
    if total > prev:
        count += 1
    prev = total
count

1065

---
## Day 2

### part 1

In [4]:
data = get_input_data(2, year=2021)
data

data = data.split('\n')

x = 0
y = 0

for step in data:
    direction, val = step.split()
    val = int(val)
    
    if direction == 'forward':
        x += val
    elif direction == 'down':
        y += val
    elif direction == 'up':
        y -= val
print(x * y)

1855814


```
down X increases your aim by X units.
up X decreases your aim by X units.
forward X does two things:
It increases your horizontal position by X units.
It increases your depth by your aim multiplied by X.
```

### part 2

In [5]:
x = 0
y = 0
aim = 0

for step in data:
    direction, val = step.split()
    val = int(val)
    
    if direction == 'forward':
        x += val
        y += val * aim
    elif direction == 'down':
        aim += val
    elif direction == 'up':
        aim -= val
print(x * y)

1845455714


---
## Day 3

In [6]:
from collections import Counter

In [7]:
data = get_input_data(3, year=2021)
data = data.split('\n')


### part 1

In [8]:
gamma = ''
epsilon = ''

for digits in zip(*data):
    counts = Counter(digits)
    (g, _), (e, _) = counts.most_common()
    gamma += g
    epsilon += e

result = int(gamma, 2) * int(epsilon, 2)
result

2498354

In [9]:
def filter_data(data, position, value):
    return [x for x in data if x[position] == value]

### Part 2

In [10]:
result = data[:]

for position in range(len(data[0])):
    digits = [x[position] for x in result]
    counts = Counter(digits)
    (oxygen, oxygen_count), (scrubber, scrubber_count) = counts.most_common()
    if oxygen_count == scrubber_count:
        oxygen = '1'
    result = filter_data(result, position, oxygen)
    if len(result) == 1:
        break

o = int(result[0], 2)
o


3921

In [11]:
result = data[:]

for position in range(len(data[0])):
    digits = [x[position] for x in result]
    counts = Counter(digits)
    (oxygen, oxygen_count), (scrubber, scrubber_count) = counts.most_common()
    if oxygen_count == scrubber_count:
        scrubber = '0'
    result = filter_data(result, position, scrubber)
    if len(result) == 1:
        break

s = int(result[0], 2)
o * s


3277956

---

## Day 4
### Part 1

In [12]:
data = get_input_data(4, year=2021)


In [13]:
# Process data
numbers, *boards = data.split('\n\n')

numbers = list(map(int, numbers.split(',')))
boards = np.array([[list(map(int, y.split()))
                    for y in x.split('\n')]
                   for x in boards])


In [14]:
def mark_results(boards, results, number):
    results[boards == number] = 1
    return results

def check_winner(boards, results):
    full_row = results.all(axis=-1)
    full_column = np.transpose(results, axes=(0, 2, 1)).all(axis=-1)

    if full_row.any():
        mask = full_row.any(axis=-1)
        return boards[mask], results[mask]
    
    if full_column.any():
        mask = full_column.any(axis=-1)
        return boards[mask], results[mask]

    return None

def sum_of_unmarked(board, result):
    return board[result == 0].sum()


In [15]:
results = np.zeros(boards.shape)
score = 0

for n in numbers:
    results = mark_results(boards, results, n)
    winner = check_winner(boards, results)
    if winner:
        board, mask = winner
        score = sum_of_unmarked(board, mask) * n
        break

score


2745

### Part 2

In [16]:
def remove_board(boards, results, objs):
    for obj in objs:
        mask = (boards == obj).all(axis=(1, 2))
        boards = np.delete(boards, mask, axis=0)
        results = np.delete(results, mask, axis=0)
    return boards, results

In [17]:
boards_ = boards.copy()
results = np.zeros(boards_.shape)
score = 0

for n in numbers:
    results = mark_results(boards_, results, n)
    winner = check_winner(boards_, results)
    if winner:
        board, mask = winner
        if len(boards_) > 1:
            boards_, results = remove_board(boards_, results, board)
        else:
            score = sum_of_unmarked(board, mask) * n
            break

score

6594

---

## Day 5
### Part 1

In [18]:
data = """0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2"""


> For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

In [19]:
def process_points(points):
    points = [tuple(map(int, x.split(','))) for x in points]
    return points

def process_data(data):
    results = []
    for line in data.split('\n'):
        points = process_points(line.split(' -> '))
        results.append(points)
    return results

def filter_horizontal_vertical(coords):
    results = []
    for coord in coords:
        (x1, y1), (x2, y2) = coord
        if (x1 == x2) or (y1 == y2):
            results.append(coord)
    return results


def generate_diagram(arr):
    arr = arr.astype(np.int32).astype(np.object0)
    arr[arr == 0] = '.'
    result = '\n'.join([''.join(x) for x in arr.astype(str)])
    return result


In [20]:
data = get_input_data(5, year=2021)

In [21]:
coords = process_data(data)
coords = filter_horizontal_vertical(coords)

In [22]:
x_dim = np.array(coords)[:,:,0].max() + 1
y_dim = np.array(coords)[:,:,1].max() + 1

A = np.zeros((y_dim, x_dim))

for (x1, y1), (x2, y2) in coords:

    # Vertical Lines
    if x1 == x2:
        y1, y2 = sorted([y1, y2])
        for y in range(y1, y2+1):
            A[y, x1] += 1

    # Horizontal lines
    else:
        x1, x2 = sorted([x1, x2])
        for x in range(x1, x2+1):
            A[y1, x] += 1

(A > 1).sum()

4655

### Part 2

In [23]:
coords = process_data(data)

In [24]:
max_dim = np.array(coords).max() + 1
A = np.zeros((max_dim, max_dim))

for (x1, y1), (x2, y2) in coords:

    # Vertical Lines
    if x1 == x2:
        y1, y2 = sorted([y1, y2])
        for y in range(y1, y2+1):
            A[y, x1] += 1

    # Horizontal lines
    elif y1 == y2:
        x1, x2 = sorted([x1, x2])
        for x in range(x1, x2+1):
            A[y1, x] += 1
    
    # Diagonal lines
    else:
        m = (y2 - y1) / (x2 - x1)
        if m < 0:
            x_start = max([x1, x2])
            x_end = min([x1, x2])
            y_start = min([y1, y2])
            steps = abs(x_end - x_start)

            for i in range(steps + 1):
                x = x_start - i
                y = y_start + i
                A[x, y] += 1
        else:
            x_start = min([x1, x2])
            x_end = max([x1, x2])
            y_start = min([y1, y2])
            y_end = max([y1, y2])
            steps = abs(x_end - x_start)

            for i in range(steps + 1):
                x = x_start + i
                y = y_start + i
                A[x, y] += 1

(A > 1).sum()

20496

--- 
## Day 6
### Part 1

In [25]:
data = '3,4,3,1,2'
data = np.array(list(map(int, data.split(','))))

In [26]:
def decrease_timers(data):
    data -= 1
    return data

def check_for_zero(data):
    mask = data == 0
    if mask.any():
        return mask
    return None

def reset_zero_timers(data, mask, timer=7):
    data[mask] = timer
    return data

def add_new_laternfish(data, mask, timer=9):
    n = mask.sum()
    data = np.append(data, np.ones(n) * timer)
    return data
    

In [27]:
data = get_input_data(6, year=2021)
data = np.array(list(map(int, data.split(','))))

In [28]:
%%time

fish = data.copy()

for i in range(80):
    zeros = check_for_zero(fish)
    if zeros is not None:
        fish = reset_zero_timers(fish, zeros)
        fish = add_new_laternfish(fish, zeros)
    fish = decrease_timers(fish)

len(fish)

CPU times: user 20.2 ms, sys: 10.4 ms, total: 30.6 ms
Wall time: 29.2 ms


386755

### Part 2

The previous niave and extremely inefficient approach does not work (I tried - it hits a serious bottle neck around the 170th loop 😆).
So let's try keeping tabs of counts with a `dict`.

In [29]:
from collections import Counter, defaultdict

def update_counts(counts):
    new_counts = {}
    
    for i in range(1, len(counts)):
        new_counts[i - 1] = counts[i]
    new_counts[8] = counts[0]
    new_counts[6] += counts[0]

    return new_counts

In [30]:
%%time

fish = data.copy()

counts = {i: 0 for i in range(9)}
counts.update(Counter(fish))

for i in range(256):
    counts = update_counts(counts)

sum(counts.values())

CPU times: user 344 µs, sys: 68 µs, total: 412 µs
Wall time: 411 µs


1732731810807

## Day 7

In [31]:
data = '16,1,2,0,4,2,7,1,2,14'
data = np.array(list(map(int, data.split(','))), dtype=np.int64)
data

array([16,  1,  2,  0,  4,  2,  7,  1,  2, 14])

### Part 1

In [32]:
data = get_input_data(7, year=2021)
data = np.array(list(map(int, data.split(','))), dtype=np.int64)

In [33]:
%%time

m = np.abs(data - data.reshape(-1, 1)).sum(1)

position = data[m.argmin()]
min_fuel = m.min()

print(min_fuel)
print(position)

336701
323
CPU times: user 5.64 ms, sys: 8.32 ms, total: 14 ms
Wall time: 9.68 ms


### Part 2

In [34]:
%%time

dists = np.abs(data - np.arange(data.max()+1).reshape(-1, 1))

results = []

for row in dists:
    results.append(sum([np.arange(x+1).sum() for x in row]))

min(results)


CPU times: user 2.89 s, sys: 25.8 ms, total: 2.92 s
Wall time: 2.91 s


95167302

---
## Day 8


```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
 ```

In [35]:
data = '''be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce'''


### Part 1

In [36]:
def process_line(line):
    signals, digits = line.split(' | ')
    return signals.split(), digits.split()

def get_sequence_lens(digits):
    return list(map(len, digits))

In [37]:
data = get_input_data(8, year=2021)

In [38]:
%%time

unique_elements = [2, 3, 4, 7]
count = 0

for line in data.split('\n'):
    signals, digits = process_line(line)
    digit_lens = get_sequence_lens(digits)
    count += np.isin(digit_lens, unique_elements).sum()

count

CPU times: user 5.92 ms, sys: 1.39 ms, total: 7.31 ms
Wall time: 6.04 ms


245

### Part 2

First find signals or digits of length 2, 3, 4, or 7 to map to the correct digits 1, 7, 4 & 8 respectively

In [39]:
def deduce_digit(digit, deduced_from, d, sequences, offset=0):
    if d.get(digit) is not None:
        return d
    keys = {k: d.get(k, set()) for k in deduced_from}
    expected_length = expected_lengths.get(digit)
    
    for element in sequences:
        if len(element) == expected_length:
            for key, val in keys.items():
                set_ = set(element)
                diff = set_ - val
                if len(diff) + len(val) - offset == expected_length:
                    d[digit] = set_
                    return d
    return d

def deduce_digit_from_remaining(digit, d, sequences):
    if d.get(digit) is not None:
        return d
    
    seen = d.values()
    expected_length = expected_lengths.get(digit)

    for element in sequences:
        if len(element) == expected_length:
            key = set(sorted(element))
            if key not in seen:
                d[digit] = key
                return d
    return d

def get_segment_map(sequences):
    d = {}

    # Digits 1, 4, 7 & 8 can be easily deduced due to their unique number of segments
    d = deduce_digit(1, [None], d, sequences)
    d = deduce_digit(4, [None], d, sequences)
    d = deduce_digit(7, [None], d, sequences)
    d = deduce_digit(8, [None], d, sequences)

    # if 1 or 7 is in the map, we can deduce 3
    # 3 will either include all the segments from 1 plus exactly 3 more segments
    # or 3 will include all the segments from 7 plus exaclty 2 more segments
    d = deduce_digit(3, [1, 7], d, sequences)

    # 9 can be deduced if 4 is known
    d = deduce_digit(9, [4], d, sequences)

    # Middle segment can be deduced from the the segments: 4 ∪ 3 - 1
    middle_segment = d[4] & d[3] - d[1]

    # 0 can be deduced from 8 - middle_segment
    d[0] = d[8] - middle_segment

    # 6  can be deduced as the last remaining seq of length 6 which hasn't been seen
    d = deduce_digit_from_remaining(6, d, sequences)
    
    # 5 can be deduced from 6
    d = deduce_digit(5, [6], d, sequences, offset=1)
    
    # 2  can be deduced as the last remaining seq of length 5 which hasn't been seen
    d = deduce_digit_from_remaining(2, d, sequences)
    
    # Swap keys and values
    d = {tuple(sorted(v)): k for k, v in d.items()}
    return d

def get_number_from_segment_map(digits, segment_map):
    out = ''
    for digit in digits:
        key = tuple(sorted(digit))
        out += str(segment_map.get(key))
    return int(out)

In [40]:
%%time

unique_elements = {2: 1, 3: 7, 4: 4, 7: 8}
expected_lengths = {0: 6, 1: 2, 2: 5, 3: 5, 4: 4, 5: 5, 6: 6, 7: 3, 8: 7, 9: 6}

result = 0

for line in data.split('\n'):
    signals, digits = process_line(line)
    segment_map = get_segment_map(signals + digits)
    result += get_number_from_segment_map(digits, segment_map)
result

CPU times: user 7.31 ms, sys: 100 µs, total: 7.41 ms
Wall time: 7.36 ms


983026

---
## Day 9
### Part 1

In [41]:
data = """2199943210
3987894921
9856789892
8767896789
9899965678"""

data = [list(map(int, x)) for x in data.split('\n')]
data

[[2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
 [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
 [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
 [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
 [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]]

In [42]:
def get_udlr(data, i, j):
    center = data[i][j]
    up = down = left = right = np.inf
    
    if i > 0:
        try:
            up = data[i-1][j]
        except IndexError:
            pass
    if j > 0:
        left = data[i][j-1]
    
    try:
        down = data[i+1][j]
    except IndexError:
        pass

    try:
        right = data[i][j+1]
    except IndexError:
        pass
    return center, (up, down, left, right)

def is_low_point(val, adj_points):
    return val < min(adj_points)

def calculate_risk(low_points):
    return sum(low_points) + len(low_points)


In [43]:
data = get_input_data(9, year=2021)
data = [list(map(int, x)) for x in data.split('\n')]

In [44]:
%%time

low_points = []
low_point_coords = []

for i, row in enumerate(data):
    for j, col in enumerate(row):
        val, adj_points = get_udlr(data, i, j)
        if is_low_point(val, adj_points):
            low_points.append(val)
            low_point_coords.append((i, j))

calculate_risk(low_points)

CPU times: user 17.2 ms, sys: 653 µs, total: 17.8 ms
Wall time: 18 ms


480

### Part 2

Looks like [flood fill](https://en.wikipedia.org/wiki/Flood_fill) can be applied here.
We've already captured the coords of low points in part 1, so lets iterate over them and run an adapted flood fill to find the basins.

In [45]:
def within_bounds(coord, data):
    n_rows = len(data)
    n_cols = len(data[0])
    x, y = coord
    if (0 <= x < n_rows) and (0 <= y < n_cols):
        return True
    return False

def get_answer(basins, n=3):
    return np.product(sorted(basins, reverse=True)[:n])

From [Wikipedia](https://en.wikipedia.org/wiki/Flood_fill)

```
Flood-fill (node):
  1. Set Q to the empty queue or stack.
  2. Add node to the end of Q.
  3. While Q is not empty:
  4.   Set n equal to the first element of Q.
  5.   Remove first element from Q.
  6.   If n is Inside:
         Set the n
         Add the node to the west of n to the end of Q.
         Add the node to the east of n to the end of Q.
         Add the node to the north of n to the end of Q.
         Add the node to the south of n to the end of Q.
  7. Continue looping until Q is exhausted.
  8. Return.
 ```

In [46]:
%%time

basins = []

for i, j in low_point_coords:
    to_check = []
    to_check.append((i, j))
    seen = []
    size = 0
    depth = 0
    
    while len(to_check) > 0:
        x, y = to_check.pop(0)
        seen.append((x, y))
        val = data[x][y]
        
        if val < 9:
            size += 1
            depth += val

            for adj_xy in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
                if (within_bounds(adj_xy, data)
                    and adj_xy not in seen
                    and adj_xy not in to_check):
                    to_check.append(adj_xy)  
    basins.append(size)

get_answer(basins)

CPU times: user 36.8 ms, sys: 1.55 ms, total: 38.3 ms
Wall time: 37 ms


1045660

---

## Day 25

In [47]:
def get_move_mask(arr, direction, axis):
    if direction not in ('>', 'v'):
        raise ValueError('direction must be one of [">", "v"]')
    return (arr == direction) & (np.roll(arr, -1, axis=axis) == '.')


def shift(arr, direction, axis):
    mask = get_move_mask(arr, direction, axis)
    arr = np.where(np.roll(mask, 1, axis=axis), np.roll(arr, 1, axis=axis), arr)
    arr[mask] = '.'
    return arr


def step_shift(arr):
    arr = shift(arr, '>', 1)
    arr = shift(arr, 'v', 0)
    return arr


def arrays_equal(arr1, arr2):
    return np.all(arr1 == arr2)


### Part 1

In [48]:
data = get_input_data(25, year=2021)
A = np.array([list(x) for x in data.split('\n')])
valid_moves = True
step = 1

while valid_moves:
    A_new = step_shift(A)
    valid_moves = not arrays_equal(A, A_new)
    if valid_moves:
        A = A_new
        step += 1

step

520